In [2]:
import pymongo
import bson
import utm
import pandas as pd
from pymongo import MongoClient

with open('/Users/mac/Desktop/dump3/test/buildings.bson','rb') as f:
    buildings = bson.decode_all(f.read())
with open('/Users/mac/Desktop/dump3/test/aps.bson','rb') as f:
    aps = bson.decode_all(f.read())
with open('/Users/mac/Desktop/dump3/test/wifi.bson','rb') as f:
    wifi = bson.decode_all(f.read())
with open('/Users/mac/Desktop/dump3/test/paths.bson','rb') as f:
    paths = bson.decode_all(f.read())


aps_df = pd.DataFrame(aps)
buildings_df = pd.DataFrame(buildings)
wifi_df = pd.DataFrame(wifi)
paths_df = pd.DataFrame(paths)
buildings_NC_df = buildings_df[(buildings_df['CampusCode']=='North Campus')
                               &(buildings_df['GeographicCampus']=='North Campus')]

In [3]:
list_path = []
list_path_full = []
for path in paths:
    list_temp = []
    list_temp_full = []
    for temp in path['readings']:
        point = temp['connectedAP']['apName']
        try:
            lat = next((l for l in aps if l['apName'] == point), None)['GIS']['x']
            lon = next((l for l in aps if l['apName'] == point), None)['GIS']['y']
        except TypeError:
            pass
        if (lat,lon) not in list_temp:
            list_temp.append((lat,lon))
            list_temp_full.append(temp)
    list_path.append(list_temp)
    list_path_full.append(list_temp_full)

path_new = [t for t in list_path if len(t) > 1]

In [4]:
path_new_full = [t for t in list_path_full if len(t) > 0]

In [5]:
list_path_df = []
for path in path_new_full:
    df1 = pd.DataFrame(path)
    df1['lon_gps'] = pd.Series(0,index = df1.index)
    df1['lat_gps'] = pd.Series(0,index = df1.index)
    df1['lon_utm'] = pd.Series(0,index = df1.index)
    df1['lat_utm'] = pd.Series(0,index = df1.index)
    i = 0
    while i < len(df1.index):
        tmp1 = df1.loc[i,'connectedAP']['apName']
        tmp2 = aps_df['apName']==tmp1
        tmp3 = aps_df[tmp2]
        if tmp3.size > 0:
            tmp4 = tmp3.iloc[0]
            tmp5 = tmp4['GIS']
            #df1.loc[i,'x'] = utm.from_latlon(float(tmp5['y']),float(tmp5['x']))[0]
            df1.loc[i,'lon_gps'] = tmp5['x']
            df1.loc[i,'lat_gps'] = tmp5['y']
            df1.loc[i,'lon_utm'] = utm.from_latlon(float(tmp5['y']),float(tmp5['x']))[0]
            df1.loc[i,'lat_utm'] = utm.from_latlon(float(tmp5['y']),float(tmp5['x']))[1] 
        i = i + 1
    list_path_df.append(df1)

In [6]:
x_min = 275766
x_max = 278141
y_min = 4685200
y_max = 4687178
trip = []
for path_tmp in list_path_df:
        if (min(path_tmp['lon_utm'])>x_min)&(max(path_tmp['lon_utm'])<x_max)&(min(path_tmp['lat_utm'])>y_min)&(max(path_tmp['lat_utm'])<y_max):
            trip.append(path_tmp)

In [7]:
list_trip_df = []
for path_df in trip:
    if len(path_df.index)>1:
        list_trip_df.append(path_df)

In [18]:
list_trip_df[0]

,_id,connectedAP,ip,mac,rssi,session,signalNoiseRatio,source,ssid,timestamp,user,lon_gps,lat_gps,lon_utm,lat_utm
0,5968f5b278c2782be07cd482,"{'radioName': None, 'apName': 'DC-1F48-S'}",35.2.139.73,557b4c5d73b1d4e9d61b4239e4606b9f1de67c526086b5...,None,{'id': '5fc7b292f1bd31b2ff149dc28e702a120cb24f...,None,RADIUS,MWireless,2017-07-14 15:47:05,{'uniqname': 'fdc1453c7ad1c2bb542d28b9cdd13486...,-83.71579239,42.29114972,276104.658225,4.685675e+06
1,5968f5b078c2782be07cab12,"{'radioName': None, 'apName': 'NCCOMM-1C01'}",35.2.67.4,557b4c5d73b1d4e9d61b4239e4606b9f1de67c526086b5...,None,{'id': '05b05e2cf6bfd2cc67c0d3290c980fe8fed56f...,None,RADIUS,MWireless,2017-07-14 15:51:10,{'uniqname': 'fdc1453c7ad1c2bb542d28b9cdd13486...,-83.7174755,42.29113561,275965.839271,4.685678e+06


In [52]:
list_trip_df[947]

,_id,connectedAP,ip,mac,rssi,session,signalNoiseRatio,source,ssid,timestamp,user,lon_gps,lat_gps,lon_utm,lat_utm
0,5968f5ab78c2782be07c382e,"{'radioName': None, 'apName': 'name-117'}",35.2.169.233,bf4627dd0d0fd3608ad767e35999e86536ee33ce29255c...,None,{'id': '5123ef9a45e297d688994b7cc48a6b542d1405...,None,RADIUS,MWireless,2017-07-14 16:01:07,{'uniqname': 'ea79a0ba6d078d6c7cc71c07c54ad50d...,-83.70915968,42.29306899,276658.297501,4.685871e+06
1,5968f59f78c2782be07b5ad2,"{'radioName': None, 'apName': 'UMTRI-OUTDOOR-W'}",35.2.18.103,bf4627dd0d0fd3608ad767e35999e86536ee33ce29255c...,None,{'id': 'a076c372944067b8c627d24a601cbd7d609325...,None,RADIUS,MWireless,2017-07-14 16:18:56,{'uniqname': 'ea79a0ba6d078d6c7cc71c07c54ad50d...,-83.70320147,42.29804525,277167.082093,4.686408e+06


In [8]:
%matplotlib
import geopandas as gpd
import glob
from itertools import tee
import networkx as nx
import pandas as pd
import numpy as np
from shapely.geometry import Point, Polygon, LineString
import matplotlib.pyplot as plt
import matplotlib
import json
import os
from collections import defaultdict

Roads = gpd.read_file("/Users/mac/Desktop/utm_test/Roads_utm2.shp")
Buildings = gpd.read_file("/Users/mac/Desktop/N-N_buildings/N-N_Buildings.shp")

def pairwise(line_coords):
    a,b = tee(line_coords, 2)
    next(b, None)
    return zip(a,b) #return (p1,p2), (p2,p3)...

def create_g(Geo_dataFrame):
    G = nx.DiGraph()
    for line in Geo_dataFrame.geometry:
       seq = pairwise(line.coords)
       for p1, p2 in seq:
           G.add_edge(p1, p2)
    return G

#Euclidean distance
def dist(n0, n1):
    x1, y1 = n0
    x2, y2 = n1
    distance = np.sqrt((x2-x1)**2 + (y2-y1)**2)
    return distance

def closest_node(sub_graph, start, end): 
    start_delta = float("inf")
    end_delta = float("inf")
    for n in sub_graph.nodes():
        s_dist = dist(start, n)
        e_dist = dist(end, n) 
        if s_dist < start_delta:
            nn_start = n
            start_delta = s_dist
        if e_dist < end_delta:
            nn_end = n
            end_delta = e_dist
    return (nn_start, nn_end)

def add_attr(sub_graph):
    for n0, n1 in sub_graph.edges_iter():
        distance = dist(n0, n1)
        sub_graph[n0][n1]["dist"] = distance

def shortest_dist(sub_graph, nn_start, nn_end):
    shortest_distance = nx.shortest_path_length(sub_graph, source= nn_start ,target= nn_end, weight= "dist")
    return shortest_distance
def shortest_path(sub_graph, nn_start, nn_end):
    shortest_path = nx.shortest_path(sub_graph, source= nn_start, target= nn_end, weight= "dist")
    return shortest_path
def calculate_distance(origin_coords, dest_coords, graph):
    nn_start, nn_end = closest_node(graph, origin_coords, dest_coords)
    shortest_distance = shortest_dist(graph, nn_start, nn_end)
    return shortest_distance

def calculate_path(origin_coords, dest_coords, graph):
    nn_start, nn_end = closest_node(graph, origin_coords, dest_coords)
    optimal_path = shortest_path(graph, nn_start, nn_end)
    return optimal_path

#convert road into graph
G = create_g(Roads)
sg = list(nx.connected_component_subgraphs(G.to_undirected()))[0]
add_attr(sg) #add distance attribute

Using matplotlib backend: MacOSX


In [9]:
columns = ['x','y', 'tripID']
df_tripline = pd.DataFrame(columns=columns)
df_tripline = df_tripline.fillna(0)

In [10]:
j = 0
while j < len(list_trip_df):
    list_x = []
    list_y = []
    test = list_trip_df[j]
    for i in test.index[test.index < len(test.index)-1]:
        O = test.loc[i,'lon_utm'],test.loc[i,'lat_utm']
        D = test.loc[i+1,'lon_utm'],test.loc[i+1,'lat_utm']
        line = calculate_path(O, D, sg)
        x = [item[0] for item in line]
        y = [item[1] for item in line]
        list_x = list_x + x
        list_y = list_y + y
    list_xy = {'x':x, 'y':y}
    df = pd.DataFrame(list_xy)
    df['tripID'] = pd.Series(j, index = range(0, 0+ len(list_x)))
    df_tripline = pd.concat([df_tripline,df])
    j = j + 1

In [31]:
df_tripline[df_tripline.tripID==2]

,x,y,tripID,geometry
0,276282.745669,4.685569e+06,2,POINT (276282.745669152 4685569.058503695)
1,276316.089766,4.685573e+06,2,POINT (276316.0897664274 4685573.117191978)
2,276331.837844,4.685578e+06,2,POINT (276331.8378438998 4685577.834487774)
3,276349.395772,4.685579e+06,2,POINT (276349.3957720192 4685578.913794021)
4,276358.260534,4.685581e+06,2,POINT (276358.2605344877 4685581.113274442)
5,276368.876515,4.685585e+06,2,POINT (276368.8765153082 4685584.665581277)
6,276380.318801,4.685591e+06,2,POINT (276380.3188006296 4685590.833971908)
7,276391.467129,4.685599e+06,2,POINT (276391.4671289499 4685598.6997734)
8,276399.249341,4.685607e+06,2,POINT (276399.2493410392 4685606.698973543)
9,276404.546381,4.685613e+06,2,POINT (276404.5463806217 4685612.749905832)


In [11]:
def pt_to_line(group):
    if len(group) != 1:
        coords = group.geometry.apply(lambda x: x.coords[0]).tolist()
        line = LineString(coords)
        return line

In [12]:
from geopandas import GeoDataFrame

# Zip the coordinates into a point object and convert to a GeoDataFrame
geometry = [Point(xy) for xy in zip(df_tripline.x, df_tripline.y)]
df_line = GeoDataFrame(df_tripline, geometry=geometry)

In [13]:
df2 = df_line.groupby(['tripID'])['geometry'].apply(lambda x: pt_to_line(x))

In [14]:
df2.to_frame()

,geometry
tripID,
0,LINESTRING (276282.745669152 4685569.058503695...
1,LINESTRING (276282.745669152 4685569.058503695...
2,LINESTRING (276282.745669152 4685569.058503695...
3,"LINESTRING (275915.8863654568 4685580.441264, ..."
4,LINESTRING (276282.745669152 4685569.058503695...
5,LINESTRING (276385.5370933599 4685979.49195418...
6,LINESTRING (276172.4678323958 4686102.13656688...
7,LINESTRING (276282.745669152 4685569.058503695...
8,None


In [51]:
df_tmp = df2.to_frame()
df_tmp[df_tmp.index==947]

,geometry
tripID,
947,LINESTRING (276577.2746662971 4685842.89102640...


In [48]:
df_tmp

,geometry
tripID,
0,LINESTRING (276282.745669152 4685569.058503695...
1,LINESTRING (276282.745669152 4685569.058503695...
2,LINESTRING (276282.745669152 4685569.058503695...
3,"LINESTRING (275915.8863654568 4685580.441264, ..."
4,LINESTRING (276282.745669152 4685569.058503695...
5,LINESTRING (276385.5370933599 4685979.49195418...
6,LINESTRING (276172.4678323958 4686102.13656688...
7,LINESTRING (276282.745669152 4685569.058503695...
8,None


In [15]:
df2.dropna()

tripID
0       LINESTRING (276282.745669152 4685569.058503695...
1       LINESTRING (276282.745669152 4685569.058503695...
2       LINESTRING (276282.745669152 4685569.058503695...
3       LINESTRING (275915.8863654568 4685580.441264, ...
4       LINESTRING (276282.745669152 4685569.058503695...
5       LINESTRING (276385.5370933599 4685979.49195418...
6       LINESTRING (276172.4678323958 4686102.13656688...
7       LINESTRING (276282.745669152 4685569.058503695...
9       LINESTRING (276361.9571860341 4685813.46118393...
10      LINESTRING (275881.6410342745 4685697.07753422...
11      LINESTRING (276282.745669152 4685569.058503695...
12      LINESTRING (276385.5370933599 4685979.49195418...
13      LINESTRING (276282.745669152 4685569.058503695...
14      LINESTRING (276282.745669152 4685569.058503695...
15      LINESTRING (276361.2562348639 4685805.74831239...
16      LINESTRING (275915.8863654568 4685580.441264, ...
17      LINESTRING (276361.9571860341 4685813.46118393...
18     

In [16]:
geo_df3 = GeoDataFrame(df2, geometry='geometry')

In [17]:
geo_df4 = geo_df3.dropna()
geo_df4

,geometry
tripID,
0,LINESTRING (276282.745669152 4685569.058503695...
1,LINESTRING (276282.745669152 4685569.058503695...
2,LINESTRING (276282.745669152 4685569.058503695...
3,"LINESTRING (275915.8863654568 4685580.441264, ..."
4,LINESTRING (276282.745669152 4685569.058503695...
5,LINESTRING (276385.5370933599 4685979.49195418...
6,LINESTRING (276172.4678323958 4686102.13656688...
7,LINESTRING (276282.745669152 4685569.058503695...
9,LINESTRING (276361.9571860341 4685813.46118393...


In [50]:
geo_df4.iloc[857]

geometry    LINESTRING (276577.2746662971 4685842.89102640...
Name: 947, dtype: object

In [53]:
geo_df4.iloc[178]

geometry    LINESTRING (277168.7532996696 4686352.10570142...
Name: 200, dtype: object

In [568]:
geo_df4.to_file('/Users/mac/Desktop/df_line.shp', driver='ESRI Shapefile')

In [1]:
geo_df4[0]

NameError: name 'geo_df4' is not defined

In [567]:
geo_df4.plot()

################################################

In [543]:
df3 = GeoDataFrame(df2, geometry='geometry')

TypeError: Input geometry column must contain valid geometry objects.

In [530]:
df_line[df_line.tripID == 8]

,x,y,tripID,geometry
0,276361.957186,4.685813e+06,8,POINT (276361.9571860341 4685813.46118393)


In [531]:
def pt_to_line(group):
    if len(group) != 1:
        coords = group.geometry.apply(lambda x: x.coords[0]).tolist()
        return coords

In [532]:
# Aggregate these points with the GroupBy
df_line2 = df_line.groupby('tripID').apply(pt_to_line)

In [534]:
df_line2.rename(columns={0:"geometry"}, inplace= True)
df_line2.

tripID
0       [(276282.745669152, 4685569.058503695), (27591...
1       [(276282.745669152, 4685569.058503695), (27591...
2       [(276282.745669152, 4685569.058503695), (27631...
3       [(275915.8863654568, 4685580.441264), (275907....
4       [(276282.745669152, 4685569.058503695), (27591...
5       [(276385.53709335986, 4685979.491954189), (276...
6       [(276172.4678323958, 4686102.136566886), (2758...
7       [(276282.745669152, 4685569.058503695), (27631...
8                                                    None
9       [(276361.95718603407, 4685813.46118393), (2763...
10      [(275881.6410342745, 4685697.077534226), (2758...
11      [(276282.745669152, 4685569.058503695), (27631...
12      [(276385.53709335986, 4685979.491954189), (276...
13      [(276282.745669152, 4685569.058503695), (27591...
14      [(276282.745669152, 4685569.058503695), (27591...
15      [(276361.25623486395, 4685805.748312398), (276...
16      [(275915.8863654568, 4685580.441264), (275907....
17     

In [524]:
df_line2.apply(lambda x: LineString(x))

tripID
0       LINESTRING (276282.745669152 4685569.058503695...
1       LINESTRING (276282.745669152 4685569.058503695...
2       LINESTRING (276282.745669152 4685569.058503695...
3       LINESTRING (275915.8863654568 4685580.441264, ...
4       LINESTRING (276282.745669152 4685569.058503695...
5       LINESTRING (276385.5370933599 4685979.49195418...
6       LINESTRING (276172.4678323958 4686102.13656688...
7       LINESTRING (276282.745669152 4685569.058503695...
8                                GEOMETRYCOLLECTION EMPTY
9       LINESTRING (276361.9571860341 4685813.46118393...
10      LINESTRING (275881.6410342745 4685697.07753422...
11      LINESTRING (276282.745669152 4685569.058503695...
12      LINESTRING (276385.5370933599 4685979.49195418...
13      LINESTRING (276282.745669152 4685569.058503695...
14      LINESTRING (276282.745669152 4685569.058503695...
15      LINESTRING (276361.2562348639 4685805.74831239...
16      LINESTRING (275915.8863654568 4685580.441264, ...
17     

In [526]:
df_line2.rename(columns={0:"geometry"}, inplace= True)

In [527]:
df_line2

,geometry
tripID,
0,"[(276282.745669152, 4685569.058503695), (27591..."
1,"[(276282.745669152, 4685569.058503695), (27591..."
2,"[(276282.745669152, 4685569.058503695), (27631..."
3,"[(275915.8863654568, 4685580.441264), (275907...."
4,"[(276282.745669152, 4685569.058503695), (27591..."
5,"[(276385.53709335986, 4685979.491954189), (276..."
6,"[(276172.4678323958, 4686102.136566886), (2758..."
7,"[(276282.745669152, 4685569.058503695), (27631..."
8,None


In [528]:
df_line2 = GeoDataFrame(df_line2,geometry = 'geometry')
df_line2

TypeError: Input geometry column must contain valid geometry objects.

In [515]:
list(df_line2.columns)

['geometry']

In [512]:
df_line2.rename(columns={0:"geometry"}, inplace= True)

In [516]:
df_line2

,geometry
tripID,
0,"[(276282.745669152, 4685569.058503695), (27591..."
1,"[(276282.745669152, 4685569.058503695), (27591..."
2,"[(276282.745669152, 4685569.058503695), (27631..."
3,"[(275915.8863654568, 4685580.441264), (275907...."
4,"[(276282.745669152, 4685569.058503695), (27591..."
5,"[(276385.53709335986, 4685979.491954189), (276..."
6,"[(276172.4678323958, 4686102.136566886), (2758..."
7,"[(276282.745669152, 4685569.058503695), (27631..."
8,None


In [ ]:
df_line.to_file('/Users/mac/Desktop/df_line.shp', driver='ESRI Shapefile')

In [331]:
df_line

,x,y,tripID,geometry
0,276282.745669,4.685569e+06,0,POINT (276282.745669152 4685569.058503695)
1,275915.886365,4.685580e+06,0,POINT (275915.8863654568 4685580.441264)
2,275907.279489,4.685580e+06,0,POINT (275907.279489004 4685579.709679094)
3,275898.752712,4.685578e+06,0,POINT (275898.7527117025 4685578.328514983)
4,275890.354987,4.685576e+06,0,POINT (275890.3549872548 4685576.305701092)
5,275882.134528,4.685574e+06,0,POINT (275882.134528465 4685573.652850648)
6,275879.743639,4.685597e+06,0,POINT (275879.7436392262 4685596.970890676)
7,275881.641034,4.685697e+06,0,POINT (275881.6410342745 4685697.077534226)
0,276282.745669,4.685569e+06,1,POINT (276282.745669152 4685569.058503695)
1,275915.886365,4.685580e+06,1,POINT (275915.8863654568 4685580.441264)


###################################test

In [227]:
columns = ['x','y', 'tripID']
df_tripline = pd.DataFrame(columns=columns)
df_tripline = df_tripline.fillna(0)

In [291]:
df_tmp = list_trip_df[0:3]

In [292]:
len(df_tmp)

3

In [295]:
list_x = []
list_y = []
test = df_tmp[0]
for i in test.index[test.index < len(test.index)-1]:
    O = test.loc[i,'lon_utm'],test.loc[i,'lat_utm']
    D = test.loc[i+1,'lon_utm'],test.loc[i+1,'lat_utm']
    line = calculate_path(O, D, sg)
    x = [item[0] for item in line]
    y = [item[1] for item in line]
    list_x = list_x + x
    list_y = list_y + y
list_xy = {'x':x, 'y':y}
df = pd.DataFrame(list_xy)
df['tripID'] = pd.Series(0, index = range(0, 0+ len(list_x)))

In [296]:
df_tripline = pd.concat([df_tripline,df])

In [297]:
df_tripline

,x,y,tripID
0,276282.745669,4.685569e+06,0
1,275915.886365,4.685580e+06,0
2,275907.279489,4.685580e+06,0
3,275898.752712,4.685578e+06,0
4,275890.354987,4.685576e+06,0
5,275882.134528,4.685574e+06,0
6,275879.743639,4.685597e+06,0
7,275881.641034,4.685697e+06,0


In [299]:
list_x = []
list_y = []
test = df_tmp[1]
for i in test.index[test.index < len(test.index)-1]:
    O = test.loc[i,'lon_utm'],test.loc[i,'lat_utm']
    D = test.loc[i+1,'lon_utm'],test.loc[i+1,'lat_utm']
    line = calculate_path(O, D, sg)
    x = [item[0] for item in line]
    y = [item[1] for item in line]
    list_x = list_x + x
    list_y = list_y + y
list_xy = {'x':x, 'y':y}
df = pd.DataFrame(list_xy,index=range(8, 8 + len(list_x)))
df['tripID'] = pd.Series(1, index = df.index)

In [300]:
df_tripline = pd.concat([df_tripline,df])

In [301]:
df_tripline

,x,y,tripID
0,276282.745669,4.685569e+06,0
1,275915.886365,4.685580e+06,0
2,275907.279489,4.685580e+06,0
3,275898.752712,4.685578e+06,0
4,275890.354987,4.685576e+06,0
5,275882.134528,4.685574e+06,0
6,275879.743639,4.685597e+06,0
7,275881.641034,4.685697e+06,0
8,276282.745669,4.685569e+06,1
9,275915.886365,4.685580e+06,1


In [302]:
list_x = []
list_y = []
test = df_tmp[2]
for i in test.index[test.index < len(test.index)-1]:
    O = test.loc[i,'lon_utm'],test.loc[i,'lat_utm']
    D = test.loc[i+1,'lon_utm'],test.loc[i+1,'lat_utm']
    line = calculate_path(O, D, sg)
    x = [item[0] for item in line]
    y = [item[1] for item in line]
    list_x = list_x + x
    list_y = list_y + y
list_xy = {'x':list_x, 'y':list_y}
df = pd.DataFrame(list_xy,index =range(16,16 +len(list_x)))
df['tripID'] = pd.Series(2,index = df.index)

In [41]:
x_temp = np.array(list_x)
y_temp = np.array(list_y)

np.savetxt('/Users/mac/Desktop/test_x.csv', x_temp)
np.savetxt('/Users/mac/Desktop/test_y.csv', y_temp)

In [ ]:
list_x = []
list_y = []
for test in list_trip_df:
    for i in test.index[test.index < len(test.index)-1]:
        O = test.loc[i,'lon_utm'],test.loc[i,'lat_utm']
        D = test.loc[i+1,'lon_utm'],test.loc[i+1,'lat_utm']
        line = calculate_path(O, D, sg)
        x = [item[0] for item in line]
        y = [item[1] for item in line]
        list_x = list_x + x
        list_y = list_y + y